## best score of 0.994 with data augmentation

In [1]:
import pandas as pd 
import numpy as np 

In [2]:
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
train = pd.read_csv('dataset/digit_recognizer/train.csv')
X = train.drop('label', axis=1)
y = train['label']

In [4]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(-1, 1))
X = scaler.fit_transform(X)

In [5]:
X = np.array(X).reshape(-1, 28, 28, 1)
y = to_categorical(y, num_classes=10)

## very simple layer for practice

In [6]:
model = Sequential([

    # first
    Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(28,28,1)),
    Conv2D(56, kernel_size=(3,3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.2),

    # second
    Conv2D(48, kernel_size=(3,3), activation='relu'),
    Conv2D(32, kernel_size=(3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.2),

    # third
    Flatten(),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dropout(0.4),

    #last
    Dense(10, activation='softmax')
])

In [7]:
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=7)

In [9]:
imggen = ImageDataGenerator(rotation_range=10,
            width_shift_range=0.1,
            height_shift_range=0.1,
            zoom_range=0.1)

imggen.fit(X_train)

In [10]:
pred = model.fit_generator(imggen.flow(X_train, y_train, batch_size=20), validation_data=(X_val, y_val), epochs=5, verbose=2)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/5
1890/1890 - 144s - loss: 0.4489 - accuracy: 0.8611 - val_loss: 0.0737 - val_accuracy: 0.9793
Epoch 2/5
1890/1890 - 145s - loss: 0.1581 - accuracy: 0.9554 - val_loss: 0.0488 - val_accuracy: 0.9871
Epoch 3/5
1890/1890 - 145s - loss: 0.1234 - accuracy: 0.9667 - val_loss: 0.0522 - val_accuracy: 0.9869
Epoch 4/5
1890/1890 - 149s - loss: 0.0984 - accuracy: 0.9718 - val_loss: 0.0525 - val_accuracy: 0.9876
Epoch 5/5
1890/1890 - 150s - loss: 0.0958 - accuracy: 0.9749 - val_loss: 0.0514 - val_accuracy: 0.9864


In [11]:
pred = model.fit_generator(imggen.flow(X, y, batch_size=20), epochs=5, verbose=2)

Epoch 1/5
2100/2100 - 153s - loss: 0.0836 - accuracy: 0.9781
Epoch 2/5
2100/2100 - 155s - loss: 0.0731 - accuracy: 0.9803
Epoch 3/5
2100/2100 - 156s - loss: 0.0696 - accuracy: 0.9814
Epoch 4/5
2100/2100 - 154s - loss: 0.0678 - accuracy: 0.9817
Epoch 5/5
2100/2100 - 143s - loss: 0.0643 - accuracy: 0.9832


In [12]:
test = pd.read_csv('dataset/digit_recognizer/test.csv')
X1 = test
scaler = MinMaxScaler(feature_range=(-1, 1))
X1 = scaler.fit_transform(X1)
X1 = X1.reshape(X1.shape[0], 28, 28, 1)

In [13]:
pred = model.predict(X1, batch_size=20)

In [14]:
pred = np.argmax(pred, axis=1)

In [15]:
sample = pd.read_csv('dataset/digit_recognizer/sample_submission.csv')
sample['Label'] = pred
sample.to_csv('dataset/digit_recognizer/2simple_data_aug.csv', index=False)